# Submitting PennyLane Circuits to Azure Quantum with the QDK

This notebook demonstrates end‑to‑end submission of a PennyLane `QNode` to Azure Quantum using the QDK compiler pipeline (OpenQASM 2 → QIR → job submission).

**Workflow overview:**

1. Create a `default.qubit` device and define a parameterized circuit (Hadamard + CNOT entanglement, then `RY(θ)` rotation).
2. Draw the circuit for a quick structural sanity check.
3. Export the bound PennyLane circuit to OpenQASM 2 using `qml.to_openqasm`.
4. Compile the OpenQASM source to QIR with `qdk.openqasm.compile` (selecting a target profile).
5. Submit the QIR payload to a chosen Azure Quantum target (e.g. `rigetti.sim.qvm`) with a shot count.
6. Retrieve and display measurement results returned by the job.

## 1. Prerequisites

This notebook assumes the `qdk`, Azure Quantum integration, and PennyLane packages are installed. You can install these with:

In [ ]:
%pip install "qdk[azure]" pennylane

This installs:
- The base qdk package (compiler, OpenQASM/QIR tooling)
- Azure Quantum client dependencies for submission
- The PennyLane package

After installing, restart the notebook kernel if it was already running. The following imports should run successfully:

In [ ]:
import pennylane as qml
from qdk.openqasm import compile
from qdk.azure import Workspace
from qdk import TargetProfile

## 2. Define a Simple Circuit
We'll create a small parameterized circuit destined for export.

In [ ]:
dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev)
def small_ansatz(theta):
    qml.H(0); qml.CNOT(wires=[0,1])
    qml.RY(theta, wires=1)
    return qml.expval(qml.PauliZ(1))

## 3. Draw for Inspection

In [ ]:
print(qml.draw(small_ansatz)(0.3))

## 4. Configure Azure Quantum workspace connection

To connect to an Azure workspace replace the following variables with your own values.

In [ ]:
subscription_id = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'
resource_group = 'myresourcegroup'
workspace_name = 'myworkspace'
location = 'westus'

## 5. Submitting the Circuit

In [ ]:
# Define parameters for the job submission
target_name = 'rigetti.sim.qvm'
target_profile = TargetProfile.Base
job_name = 'pennylane-job'
shots = 100

# Define parameter for the circuit
theta = 0.3

# Specify Azure Quantum workspace
workspace = Workspace(
    subscription_id=subscription_id,
    resource_group=resource_group,
    name=workspace_name,
    location=location,
)

# Compile the PennyLane circuit to QIR via OpenQASM
qasm_str = qml.to_openqasm(small_ansatz)(theta)
qir = compile(qasm_str, target_profile)

# Submit the job to Azure Quantum
target = workspace.get_targets(target_name)
job = target.submit(qir, job_name, shots)

# Print the results
results = job.get_results()
print(results)